### Trip Volume Analysis
## How has the volume of Bixi trips changed over time?

using open data from bixi's website from 2014 to 2024

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss

## Part 1 : Cleaning data

In [7]:
data_folder = '../raw_data/'
